In [1]:
import numpy as np
import torch
import torchio.transforms
from tqdm import tqdm
import os
import glob
import SimpleITK as sitk
from scipy import ndimage
import copy
import random
import time
import numpy as np
import torch
from PIL import Image
from scipy.special import comb
from torch.utils.data import Dataset


## 其他

测试CUDA

In [ ]:
#测试cuda
import torch
print(torch.__version__)
print(torch.cuda.is_available())

查看permutations文件

In [ ]:
k_permutations_path = r'datasets_3D\Rubik_cube\permutaions\permutations_hamming_max_100.npy'
K_permutations = np.load(k_permutations_path)
#可以用vscode自带的数据查看器查看

测试np.flip产生的错误step must be greater than zero

In [ ]:
x = torch.randn(10, 10)
# x[:, ::-1]
# > ValueError: step must be greater than zero
x.numpy()[:, ::-1] # works


In [ ]:

a = np.random.randn(10, 10)
a.strides
# > (80, 8)


In [ ]:
a = np.flip(a)
a.strides
# > (-80, -8) strides的涵义我收藏了一篇文章可以翻阅

x = torch.from_numpy(a)
# > ValueError: At least one stride in the given numpy array is negative, and tensors with negative strides are not currently supported. (You can probably work around this by making a copy of your array  with array.copy().) 

x = torch.from_numpy(a.copy()) # works
x.stride()
# > (10, 1)

## 测试MRI_rkb_plus_pretask中的get_item

### 测试全代码

设置假设input

In [ ]:
t1 = torch.rand(240,240,155)
print(t1.shape)

设置假设config参数值

In [ ]:
crop_size = [120, 120, 77]
flag = 'train'
num_grids_per_axis = 2
num_cubes = num_grids_per_axis ** 3
print(num_cubes)
k_permutations_path = r'datasets_3D\Rubik_cube\permutaions\permutations_hamming_max_100.npy'
K_permutations = np.load(k_permutations_path)


def会用到的一些函数（from base_rkb_pretask）

In [ ]:
def crop_3d(image, flag, crop_size): # ...（对图像执行3D裁剪的方法）
    h, w, d = crop_size[0], crop_size[1], crop_size[2]
    h_old, w_old, d_old = image.shape[0], image.shape[1], image.shape[2]

    if flag == 'train': #如果 flag 是 'train'，则进行随机裁剪。在这种情况下，生成随机的裁剪起始点 (x, y, z)，h_old-h是为了确保裁剪框不超出图像边界。
        # crop random
        x = np.random.randint(0, 1 + h_old - h) #返回一个随机数或随机数数组(指定size时)
        y = np.random.randint(0, 1 + w_old - w)
        z = np.random.randint(0, 1 + d_old - d)
    else: #如果 flag 不是 'train'，则进行中心裁剪。计算裁剪起始点 (x, y, z) 使得裁剪的区域在图像中居中。
        # crop center
        x = int((h_old - h) / 2)
        y = int((w_old - w) / 2)
        z = int((d_old - d) / 2)

    return do_crop_3d(image, x, y, z, h, w, d)

def do_crop_3d(image, x, y, z, h, w, d): #3D裁剪的辅助函数，确保整数和返回对应位置的3D块
    assert type(x) == int, x
    assert type(y) == int, y
    assert type(z) == int, z
    assert type(h) == int, h
    assert type(w) == int, w
    assert type(d) == int, d

    return image[x:x + h, y:y + w, z:z + d] #这边取的时最边界点加上crop_size的大小，解释了为何上述代码的定中心方式如此

def crop_cubes_3d(image, flag, cubes_per_side, cube_jitter_xy=3, cube_jitter_z=3): #在3D图像中裁剪多个3D立方体的方法，jitter貌似是为了防止数据连续变化留出的间隔大小，从代码里可以看出如果有jitter则再次进行3D裁切，cubes_per_side应该是每个方向有多少个cube
    h, w, d = image.shape

    patch_overlap = -cube_jitter_xy if cube_jitter_xy < 0 else 0 #patch_overlap 计算了在裁剪时可能存在的重叠区域。如果 cube_jitter_xy 是负数，则 patch_overlap 采用该值；否则，为零。

    #h_grid、w_grid 和 d_grid 计算了每个立方体的网格大小，以确保裁剪时没有重叠区域。
    # 这里主要思想应该是把切出来的3D大块划分为各个小块,这里拿笔记做做数学题吧，好久没写了。
    h_grid = (h - patch_overlap) // cubes_per_side
    w_grid = (w - patch_overlap) // cubes_per_side
    d_grid = (d - patch_overlap) // cubes_per_side
    h_patch = h_grid - cube_jitter_xy
    w_patch = w_grid - cube_jitter_xy
    d_patch = d_grid - cube_jitter_z

    cubes = []
    for i in range(cubes_per_side):
        for j in range(cubes_per_side):
            for k in range(cubes_per_side):

                p = do_crop_3d(image, #当i=0时，从0截取到第一个grid
                                i * h_grid,
                                j * w_grid,
                                k * d_grid,
                                h_grid + patch_overlap,
                                w_grid + patch_overlap,
                                d_grid + patch_overlap)

                if h_patch < h_grid or w_patch < w_grid or d_patch < d_grid: #如果有jitter，则对之前裁切好的3D块再次裁切以保留jitter大小的间隔
                    p = crop_3d(p, flag, [h_patch, w_patch, d_patch])

                cubes.append(p)

    return cubes

def rearrange(cubes, K_permutations):  # ...根据排列重新排列立方体的方法
    label = random.randint(0, len(K_permutations) - 1)
    print('label', np.array(K_permutations[label]), label)
    return np.array(cubes)[np.array(K_permutations[label])], label

def center_crop_xy(image, size): # 在image中间截一个size大小的缺口，画图很容易理解
    """CenterCrop a sample.
        Args:
            image: [D, H, W]
            label:[D, H, W]
            crop_size: the desired output size in the x-y plane
        Returns:
            out_image:[D, h, w]
            out_label:[D, h, w]
    """
    h, w, d = image.shape

    h1 = int(round((h - size[0]) / 2.)) #round() 函数将结果四舍五入为最接近的整数，确保裁剪区域的起始点是整数。
    w1 = int(round((w - size[1]) / 2.))

    image = image[h1:h1 + size[0], w1:w1 + size[1], :]
    return image

def rotate(cubes): # ...（旋转3D立方体的方法）

    # multi-hot labels
    # [8, H, W, D]
    rot_cubes = copy.deepcopy(cubes) # 创建输入立方体的深层副本，以免修改原始数据
    hor_vector = [] # 记录水平旋转的向量
    ver_vector = [] # 记录垂直旋转的向量

    for i in range(num_cubes):
        p = random.random()  # 生成一个0到1之间的随机数
        cube = rot_cubes[i] # 获取当前处理的立方体,这里debug看cube是个tensor，后续会出bug
        cube_ = cube.numpy() #设置一个numpy_arrary防止np.flip bug
        # p = 0.1 # for test
        # [H, W, D]
        if p < 1/3: # 如果 p 小于 1/3，表示进行水平旋转。将水平旋转的标志添加到 hor_vector，并沿x轴翻转180度。
            hor_vector.append(1)
            ver_vector.append(0)
            # rotate 180 along x axis
            # rot_cubes[i] = np.flip(cube, (1, 2)) # 沿x轴翻转180度,BUG ValueError: step must be greater than zero,最后输出的应该是numpy形式
            cube = np.flip(cube_, (1, 2))
            rot_cubes[i] = cube
        elif p < 2/3: #如果 p 大于等于 1/3 且小于 2/3，表示进行垂直旋转。将垂直旋转的标志添加到 ver_vector，并沿z轴翻转180度。
            hor_vector.append(0)
            ver_vector.append(1)
            # rotate 180 along z axis
            # rot_cubes[i] = np.flip(cube, (0, 1)) # 沿z轴翻转180度
            cube = np.flip(cube_, (0, 1))
            rot_cubes[i] = cube

        else: #如果 p 大于等于 2/3，不进行旋转，标志都设为0。
            hor_vector.append(0)
            ver_vector.append(0)

    return rot_cubes, hor_vector, ver_vector

def mask(cubes): # ...（对3D立方体应用掩码的方法）
    mask_vector = []
    masked_cubes = copy.deepcopy(cubes) # 创建一个输入立方体的深层副本，以免修改原始数据
    for i in range(num_cubes):
        cube = masked_cubes[i] # 获取当前处理的立方体
        if random.random() < 0.5: # 如果随机数小于0.5，应用掩码
            # mask
            mask_vector.append(1) # 记录掩码的标志为1
            R = np.random.uniform(0, 1, cube.shape) # 生成与立方体相同形状的随机数矩阵
            R = (R > 0.5).astype(np.int32) # 将大于0.5的值设置为1，否则为0，形成二值掩码
            masked_cubes[i] = cube * R # 将立方体与二值掩码相乘，进行掩码操作
        else:
            mask_vector.append(0) # 如果随机数大于等于0.5，不应用掩码，标志为0

    return masked_cubes, mask_vector

测试代码主体

In [ ]:
input_tensor = torch.Tensor(t1)
# input: [240, 240, 155]
        
input = input_tensor[:,:,0:154]
# 截去最后一层方便分块整除

if crop_size == [128, 128, 32]:
    # input: [276, 276, 74]
    input = center_crop_xy(input, [276, 276])

    # get all the num_grids **3 cubes
    all_cubes = crop_cubes_3d(
        input,
        flag=flag,
        cubes_per_side=num_grids_per_axis,
        cube_jitter_xy=10,
        cube_jitter_z=5,
    )
    print(len(all_cubes), all_cubes[0].shape)

# 这部分添加适合MRI的块大小
# MRI数据集大小[240,240,155],为了能够整除，应该截成154的depth，设crop_size=[120，120，77],之所以做的是整除原因在于抄的上面的代码也是整除
elif crop_size == [120, 120, 77]:
    # input: [276, 276, 74]
    # input = center_crop_xy(input, [276, 276])

    # get all the num_grids **3 cubes
    all_cubes = crop_cubes_3d(
        input,
        flag=flag,
        cubes_per_side=num_grids_per_axis,
        cube_jitter_xy=10,
        cube_jitter_z=5,
    )
    print(len(all_cubes), all_cubes[0].shape)

elif crop_size == [64, 64, 16]:
    # input: [140, 140, 40]
    input = ndimage.zoom(input, [140 / 320, 140 / 320, 40 / 74], order=3)

    # get all the num_grids **3 cubes
    all_cubes = crop_cubes_3d(
        input,
        flag=flag,
        cubes_per_side=num_grids_per_axis,
        cube_jitter_xy=6,
        cube_jitter_z=4,
    )

else:
    NotImplementedError("This crop size has not been configured yet")
    all_cubes = None

# 这里all_cubes还是list形式

In [ ]:

# Task1: Permutate the order of cubes
rearranged_cubes, order_label = rearrange(all_cubes, K_permutations)

print(rearranged_cubes.dtype)

In [ ]:
# Task2: Rotate each cube randomly.
rotated_cubes, hor_label, ver_label = rotate(rearranged_cubes)

# Task2: Mask each cube randomly.
masked_cubes, mask_label = mask(rotated_cubes)

print(rotated_cubes.shape) # (8,)
print(rotated_cubes[0].shape) # torch.Size([110, 110, 72])



In [ ]:
# final_cubes = np.expand_dims(np.array(masked_cubes), axis=1) #在a xis=1的位置增加一个维度，可参文章
final_cubes = masked_cubes.tolist() # 转为list方便转换为tensor
final_cubes_tensors = torch.stack(torch.from_numpy(final_cubes))


In [ ]:

print(final_cubes.shape) # (8, 1)
print(final_cubes[0].shape) # (1,)
print(final_cubes[0][0].shape) # torch.Size([110, 110, 72])
print(final_cubes_tensors.shape) # (8, 1)
print(final_cubes_tensors[0].shape) # (1,)
print(final_cubes_tensors[0][0].shape) # torch.Size([110, 110, 72])


In [ ]:
final_cubes_tensors = torch.stack(cubes)
print(final_cubes_tensors.dtype)

In [ ]:
final_cubes_list = final_cubes.tolist()
final_cubes_tensors = torch.stack(final_cubes_list)
print(final_cubes_tensors.dtype)

return的结果

In [ ]:
# 以下是4个要输出的值
print(torch.from_numpy(final_cubes.astype(np.float32)))
print(torch.from_numpy(np.array(order_label)))
print(torch.from_numpy(np.array(hor_label)).float())
print(torch.from_numpy(np.array(ver_label)).float())


### 测试rearrange函数

问题在rearrange函数，进行调试

In [2]:
# 创建包含8个随机张量的张量
num_tensors = 8
tensor_size = [110, 110, 72]

random_tensors = [torch.rand(tensor_size) for _ in range(num_tensors)]
# tensor_arry = np.array(random_tensors) # 源代码中输入的就是list，不需要转为nparrary


# 打印结果
print(len(random_tensors))
print(random_tensors[0].shape) # (1,)


8
torch.Size([110, 110, 72])


In [3]:
cubes = random_tensors
cubes_tensor = torch.stack(cubes) # 将嵌套list转为tensor增加测试
k_permutations_path = r'datasets_3D\Rubik_cube\permutaions\permutations_hamming_max_100.npy'
K_permutations = np.load(k_permutations_path)

In [4]:
label = random.randint(0, len(K_permutations) - 1)
label = 50 # 设置个label方便debug

In [5]:
choosed_permutation = K_permutations[label] # 这里挑出来是nparray类型
print(choosed_permutation)
print(len(choosed_permutation))

[2 3 0 1 5 6 7 4]
8


测试tensor排序

In [ ]:
rearrange_cubes_tensor = cubes_tensor[choosed_permutation]
# 检查是否相等
are_close = torch.allclose(rearrange_cubes_tensor[2], cubes_tensor[0])
print(are_close)
are_close = torch.allclose(rearrange_cubes_tensor[1], cubes_tensor[0])
print(are_close)

In [6]:
cubes_array = np.array(cubes) # 这句报警告

d:\Programming\anaconda3\envs\Medic_ssl\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  """Entry point for launching an IPython kernel.
d:\Programming\anaconda3\envs\Medic_ssl\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [ ]:
cubes_array

In [ ]:
cubes_rearrange =cubes_array[np.array(K_permutations[label])] # 貌似用nparray操作最合适不会报错

In [ ]:
cubes_tensor = torch.tensor(cubes)

In [ ]:
cubes_rearrange =cubes[np.array(K_permutations[label])]

In [ ]:
print('label', np.array(K_permutations[label]), label)
return np.array(cubes)[np.array(K_permutations[label])], label

In [ ]:
import torch

# 嵌套列表
nested_list = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
]

# 将嵌套列表转换为 PyTorch 张量
nested_tensor = torch.tensor(nested_list)

# 打印结果
print(nested_tensor)

## 测试VIT

In [ ]:
import torch
from vit_pytorch.vit_3d import ViT


In [ ]:

v = ViT(
    image_size = 128,          # image size
    frames = 16,               # number of frames
    image_patch_size = 16,     # image patch size
    frame_patch_size = 2,      # frame patch size
    num_classes = 1000,
    dim = 1024,
    depth = 6,
    heads = 8,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1
)

video = torch.randn(4, 3, 16, 128, 128) # (batch, channels, frames, height, width)

preds = v(video) # (4, 1000)

## 测试dataset

In [1]:
import os
import numpy as np
from tqdm import tqdm
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"
from trainers import *
import argparse


class rkbp_config:

    attr = 'class'
    # gpu_ids = [0, 1]
    gpu_ids = [0] #没有多块GPU，做个修改看能不能跑通
    benchmark = False #用于设置cudnn.benchmark
    manualseed = 666 #随机数种子
    model = 'Simple' #用于决定是否是复杂的模型，在源码中BROL和PCRL是复杂模型
    # network = 'unet_3d_rkbp' #用于base_trainer中init_model中的get_networks
    network = 'VIT_3d'
    init_weight_type = 'kaiming'
    note = "RKBPlus_240_240_155_MRI" #修改标记

    # data
    train_fold = [0, 1, 2, 3, 4] #rkb_plus_pretask.py中定义的get_luna_list中用到
    valid_fold = [5, 6]
    test_fold = [7, 8, 9]
    hu_min = -1000.0
    hu_max = 1000.0
    scale = 4
    input_size = [120, 120, 77] # 自己设置的适合切MRI尺寸的大小 # [64, 64, 16]  # [128, 128, 64]
    org_data_size = [240, 240, 155-1] #[320, 320, 74]
    train_dataset = 'MRI' #用于init_dataloader的get_dataloder_3D
    eval_dataset = 'MRI'
    im_channel = 1 #用于init_model中的get_networks
    order_class_num = 100 #这里的num之所以是100是因为最后要设置分类头对应排序的种数，如果8的阶乘全部用上就太大了，所以挑了个最大数
    k_permutations_path = r'datasets_3D\Rubik_cube\permutaions\permutations_hamming_max_100.npy'
    #k_permutation在Rubik_cube中的base_rkb_pretask中进行处理，里面存的是所有可能的排列组合顺序和对应标签顺序，比如[1,2,3,4...,6,7,8]为第0种顺序，1 到 8 的数字的排列方式有 40,320 种（8的阶乘）所以文件里存的应该是对应的最大100种，和最大1000种，可以调用出来看一下就知道了
    gaps = [6, 6, 4]
    num_grids_per_axis = 2

    # model pre-training
    train_batch = 2 #用于init_dataloader的get_dataloder_3D
    val_batch = 2
    optimizer = "adam"
    scheduler = 'StepLR_multi_step'
    learning_rate_decay = [250]#[200]
    num_workers = 1 #用于init_dataloader的get_dataloder_3D
    max_queue_size = num_workers * 4
    epochs = 1000
    save_model_freq = 50
    patience = 40
    lr = 1e-3
    loss = 'ce' #用于设定loss

    # logs
    resume = None #继续训练之前训练过的模型
    pretrained_model = None #用于trainer中的get_training_phase函数

    def display(self, logger):
        """Display Configuration values."""
        logger.info("\nConfigurations:")
        for a in dir(self):
            if not a.startswith("__") and not callable(getattr(self, a)) and not '_idx' in a:
                logger.info("{:30} {}".format(a, getattr(self, a)))
        logger.info("\n")



config = rkbp_config()
Trainer = RKBPTrainer(config)
# Trainer.train()

Experiment dir: ../checkpoints/MRI/VIT_3d_Simple_RKBPlus_240_240_155_MRI/20231205-220837
RUNDIR: ../checkpoints/MRI/VIT_3d_Simple_RKBPlus_240_240_155_MRI/20231205-220837
12/05 10:08:37 PM Simple-Train
12/05 10:08:37 PM 
Configurations:
12/05 10:08:37 PM attr                           class
12/05 10:08:37 PM benchmark                      False
12/05 10:08:37 PM epochs                         1000
12/05 10:08:37 PM eval_dataset                   MRI
12/05 10:08:37 PM gaps                           [6, 6, 4]
12/05 10:08:37 PM gpu_ids                        [0]
12/05 10:08:37 PM hu_max                         1000.0
12/05 10:08:37 PM hu_min                         -1000.0
12/05 10:08:37 PM im_channel                     1
12/05 10:08:37 PM init_weight_type               kaiming
12/05 10:08:37 PM input_size                     [120, 120, 77]
12/05 10:08:37 PM k_permutations_path            datasets_3D\Rubik_cube\permutaions\permutations_hamming_max_100.npy
12/05 10:08:37 PM learning_rate_d

In [2]:
dataset = Trainer.train_dataset

In [4]:
test = dataset[0]

In [5]:
# final_cubes
print(test[0].shape)

torch.Size([8, 1, 110, 110, 72])


In [10]:
# order_label
print(test[1].shape)
print(test[1])

torch.Size([])
tensor(19, dtype=torch.int32)


In [11]:
# hor_label
print(test[2].shape)
print(test[2])

torch.Size([8])
tensor([1., 0., 0., 0., 0., 0., 1., 0.])


In [12]:
# ver_label
print(test[3].shape)
print(test[3])

torch.Size([8])
tensor([0., 0., 1., 1., 0., 1., 0., 0.])


In [13]:
# mask_label
print(test[4].shape)
print(test[4])

torch.Size([8])
tensor([1., 0., 1., 0., 0., 1., 0., 0.])


## Unet3d到VIT3d

### 测试Vit作为encoder

In [6]:
import torch
from vit_pytorch.vit_3d import ViT

In [8]:
v = ViT(
    image_size = 128,          # image size
    frames = 16,               # number of frames
    image_patch_size = 16,     # image patch size
    frame_patch_size = 2,      # frame patch size
    num_classes = 100,
    dim = 1024,
    depth = 2,
    heads = 8,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1,
    channels = 1
)

video = torch.randn(4, 1, 16, 128, 128) # (batch, channels, frames, height, width)

preds = v(video) # (4, 100)

In [14]:
from einops import rearrange

# SSL部分dataset排列为B,C,X,Y,Z 进入之前需要重排一下顺序

# dataset读取的顺序如下
video = torch.randn(4, 1, 240, 240, 155) # (batch, channels, height, width, frames)

video_cut = torch.randn(4, 1, 110, 110, 72) # (batch, channels, height, width, frames)

print(video_cut.shape)


torch.Size([4, 1, 110, 110, 72])


In [15]:
video_cut = rearrange(video_cut, 'b c h w f -> b c f h w')
# 这里rearrange和Rearrange效果不一样，大写的是类可参vit_3d里的用法

print(video_cut.shape)


torch.Size([4, 1, 72, 110, 110])


In [16]:
v = ViT(
    image_size = 110,          # image size
    frames = 72,               # number of frames
    image_patch_size = 110,     # image patch size
    frame_patch_size = 1,      # frame patch size
    num_classes = 64,
    dim = 1024,
    depth = 1,
    heads = 8,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1,
    channels = 1
)

preds = v(video_cut)

print(preds.shape)


torch.Size([4, 100])


### 修改unet3d为vit3d

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import random
import copy
import numpy as np

In [38]:
from networks.unet3d import UNet3D_RKBP

# 测试output_fc6 = self.forward_once(cubes[i])，cubes[i] = b,1,72,110,110

# 创建类的实例
my_instance = UNet3D_RKBP()

b=2
data = torch.randn([b,1,72,110,110])

output_fc6 = my_instance.forward_once(data)
print(output_fc6.shape) # [2, 64]

torch.Size([2, 64])


#### 修改草稿

In [40]:
from vit_pytorch.vit_3d import ViT
from einops import rearrange

class VIT3D_RKBP(nn.Module):
    # the number of convolutions in each layer corresponds
    # to what is in the actual prototxt, not the intent
    def __init__(self, in_channels=1, order_n_class=100, num_cubes=8, act='relu'):
        super(VIT3D_RKBP, self).__init__()
        
        self.vit_encoder64 = ViT(
                image_size = 110,          # image size
                frames = 72,               # number of frames
                image_patch_size = 110,     # image patch size
                frame_patch_size = 1,      # frame patch size
                num_classes = 64,          # encoded dimension
                dim = 1024,
                depth = 2,
                heads = 8,
                mlp_dim = 2048,
                dropout = 0.1,
                emb_dropout = 0.1,
                channels = in_channels)
        # self.encoder = UNet3D_Encoder(in_channels=in_channels, act=act)
        self.gap = torch.nn.AdaptiveAvgPool3d(1)


        self.fc6 = nn.Sequential(
            nn.Linear(512, 64),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(64),
        )

        self.order_fc = nn.Sequential(
            nn.Linear(num_cubes * 64, 1024),
            nn.ReLU(inplace=True),

            nn.Linear(1024, 1024),
            nn.ReLU(inplace=True),

            nn.Linear(1024, order_n_class)
        )

        self.ver_rot_fc = nn.Sequential(
            nn.Linear(num_cubes * 64, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, num_cubes))

        self.hor_rot_fc = nn.Sequential(
            nn.Linear(num_cubes * 64, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, num_cubes))

        self.mask_fc = nn.Sequential(
            nn.Linear(num_cubes * 64, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, num_cubes))

        self.num_cubes = num_cubes
        self.sigmoid = torch.nn.Sigmoid()

    def forward_once(self, x):
        # x.shape[b, 1, 72, 110, 110]
        x_rearrange = rearrange(x, 'b c h w f -> b c f h w')
        # x_rearrange.shape[b, 1, 72, 110, 110] to fit vit

        logits = self.vit_encoder64(x_rearrange) # logits[b,64]
        return logits

    def forward(self, cubes):
        # [B, 8, C, X, Y, Z]
        cubes = cubes.transpose(0, 1)
        # [8, B, C, X, Y, Z]
        feats = []
        for i in range(self.num_cubes):
            output_fc6 = self.forward_once(cubes[i]) # output尺寸与输入一致
            feats.append(output_fc6) # 
            ## hor_rot_logit: [B, 1]

        feats = torch.cat(feats, 1) # feats[8, B, C, X, Y, Z]

        # order_logits: [B, K]
        order_logits = self.order_fc(feats)

        # hor_rot_logits: [B*8, 1]
        hor_rot_logits = self.sigmoid(self.hor_rot_fc(feats))
        ver_rot_logits = self.sigmoid(self.ver_rot_fc(feats))
        # mask
        mask_logits = self.sigmoid(self.mask_fc(feats))

        return order_logits, hor_rot_logits, ver_rot_logits, mask_logits

    @staticmethod
    def get_module_dicts():
        encoder_layers = ['down_tr64', 'down_tr128', 'down_tr256', 'down_tr512']
        fc_layers = ['fc6', 'order_fc', 'hor_rot_fc', 'ver_rot_fc']
        module_dict = {'encoder': encoder_layers, 'fc':fc_layers}
        return module_dict


草稿测试

In [43]:
# 创建类的实例
Vit_test = VIT3D_RKBP()

# cubes[B, 8, C, X, Y, Z]
b=2
data = torch.randn([b, 8, 1, 110, 110, 72])
print(data.shape)

torch.Size([2, 8, 1, 110, 110, 72])


In [47]:
order_logits, hor_rot_logits, ver_rot_logits, mask_logits = Vit_test(data)
print(order_logits.shape)
print(hor_rot_logits)
print(hor_rot_logits.shape)
print(ver_rot_logits)
print(ver_rot_logits.shape)
print(mask_logits)
print(mask_logits.shape)

torch.Size([2, 100])
tensor([[0.5016, 0.4935, 0.4386, 0.5066, 0.4757, 0.4933, 0.5227, 0.5233],
        [0.5043, 0.4844, 0.4336, 0.4920, 0.4751, 0.4723, 0.5265, 0.5382]],
       grad_fn=<SigmoidBackward0>)
torch.Size([2, 8])
tensor([[0.5105, 0.5606, 0.5270, 0.4956, 0.4892, 0.4640, 0.4621, 0.4769],
        [0.5268, 0.5521, 0.5229, 0.4999, 0.4748, 0.4605, 0.4508, 0.4561]],
       grad_fn=<SigmoidBackward0>)
torch.Size([2, 8])
tensor([[0.4690, 0.4631, 0.4723, 0.5256, 0.5147, 0.5237, 0.4695, 0.5019],
        [0.4800, 0.4420, 0.4793, 0.5408, 0.4969, 0.5480, 0.4416, 0.5028]],
       grad_fn=<SigmoidBackward0>)
torch.Size([2, 8])


测试MRI图像重排列

In [18]:
import torch

b = 2
all_cubes = torch.randn([b, 8, 1, 110, 110, 72])
print(all_cubes.shape)

torch.Size([2, 8, 1, 110, 110, 72])


In [19]:
all_cubes_list = []
for i in range(all_cubes.shape[1]):
    all_cubes_list.append(all_cubes[:,i,:,:,:,:])

print(all_cubes_list[0].shape)

torch.Size([2, 1, 110, 110, 72])


In [23]:
cube_jitter_xy=10
cube_jitter_z=5
jitter_xy = 2*10 
jitter_xy_cube = torch.zeros(b, 1, jitter_xy, 110, 72)
print(jitter_xy_cube.shape)
test_tensor = torch.cat((all_cubes_list[0],jitter_xy_cube),dim=2)
print(test_tensor.shape)
test_tensor2 = torch.cat((test_tensor,all_cubes_list[1]),dim=2)
print(test_tensor2.shape)


torch.Size([2, 1, 20, 110, 72])
torch.Size([2, 1, 130, 110, 72])
torch.Size([2, 1, 240, 110, 72])


In [24]:
for i in range(0,8,2):
    print(i)

0
2
4
6


In [28]:
cat_tensor_1 = []
for i in range(0,8,2):
    jitter_xy_cube = torch.zeros(b, 1, jitter_xy, 110, 72)
    # print(jitter_xy_cube.shape)
    test_tensor = torch.cat((all_cubes_list[i],jitter_xy_cube),dim=2)
    # print(test_tensor.shape)
    test_tensor2 = torch.cat((test_tensor,all_cubes_list[i+1]),dim=2)
    # print(test_tensor2.shape)
    cat_tensor_1.append(test_tensor2)

print(cat_tensor_1[0].shape)

torch.Size([2, 1, 240, 110, 72])


In [29]:
cat_tensor_2 = []
for i in range(0,4,2):
    jitter_xy_cube = torch.zeros(b, 1, 240, jitter_xy, 72)
    # print(jitter_xy_cube.shape)
    test_tensor = torch.cat((cat_tensor_1[i],jitter_xy_cube),dim=3)
    # print(test_tensor.shape)
    test_tensor2 = torch.cat((test_tensor,cat_tensor_1[i+1]),dim=3)
    # print(test_tensor2.shape)
    cat_tensor_2.append(test_tensor2)

print(cat_tensor_2[0].shape)

torch.Size([2, 1, 240, 240, 72])


In [32]:
jitter_z = 2*cube_jitter_z
jitter_z_cube = torch.zeros(b, 1, 240, 240, jitter_z)
print(jitter_xy_cube.shape)
test_tensor = torch.cat((cat_tensor_2[0],jitter_z_cube),dim=4)
print(test_tensor.shape)
catted_tensor = torch.cat((test_tensor,cat_tensor_2[1]),dim=4)
print(catted_tensor.shape)

torch.Size([2, 1, 240, 20, 72])
torch.Size([2, 1, 240, 240, 82])
torch.Size([2, 1, 240, 240, 154])


In [11]:
test_tensor_sq = torch.cat((test_tensor, test_tensor2),dim=3)
print(test_tensor_sq.shape)

torch.Size([2, 1, 220, 220, 72])


In [4]:
def bak_to_onecube(all_cubes,cube_jitter_xy=10,cube_jitter_z=5):
    all_cubes_list = []
    for i in range(all_cubes.shape[0]):
        all_cubes_list.append(all_cubes[i,:,:,:,:])
    jitter_xy = 2*cube_jitter_xy
    jitter_z = 2*cube_jitter_z
    cat_tensor_1 = []
    for i in range(0,8,2):
        jitter_xy_cube = torch.zeros(1, jitter_xy, 110, 72)
        # print(jitter_xy_cube.shape)
        test_tensor = torch.cat((all_cubes_list[i],jitter_xy_cube),dim=1)
        # print(test_tensor.shape)
        test_tensor2 = torch.cat((test_tensor,all_cubes_list[i+1]),dim=1)
        # print(test_tensor2.shape)
        cat_tensor_1.append(test_tensor2)
    cat_tensor_2 = []
    for i in range(0,4,2):
        jitter_xy_cube = torch.zeros(1, 240, jitter_xy, 72)
        # print(jitter_xy_cube.shape)
        test_tensor = torch.cat((cat_tensor_1[i],jitter_xy_cube),dim=2)
        # print(test_tensor.shape)
        test_tensor2 = torch.cat((test_tensor,cat_tensor_1[i+1]),dim=2)
        # print(test_tensor2.shape)
        cat_tensor_2.append(test_tensor2)
    jitter_z = 2*cube_jitter_z
    jitter_z_cube = torch.zeros(1, 240, 240, jitter_z)
    # print(jitter_xy_cube.shape)
    test_tensor = torch.cat((cat_tensor_2[0],jitter_z_cube),dim=3)
    # print(test_tensor.shape)
    catted_tensor = torch.cat((test_tensor,cat_tensor_2[1]),dim=3)
    # print(catted_tensor.shape)
    return catted_tensor

In [5]:
import torch

all_cubes = torch.randn([8, 1, 110, 110, 72])
print(all_cubes.shape)
catted_tensor = bak_to_onecube(all_cubes)
print(catted_tensor.shape)

torch.Size([8, 1, 110, 110, 72])
torch.Size([1, 240, 240, 154])


In [ ]:
def crop_3d(self, image, flag, crop_size): # ...（对图像执行3D裁剪的方法）
    h, w, d = crop_size[0], crop_size[1], crop_size[2]
    h_old, w_old, d_old = image.shape[0], image.shape[1], image.shape[2]

    if flag == 'train': #如果 flag 是 'train'，则进行随机裁剪。在这种情况下，生成随机的裁剪起始点 (x, y, z)，h_old-h是为了确保裁剪框不超出图像边界。
        # crop random
        x = np.random.randint(0, 1 + h_old - h) #返回一个随机数或随机数数组(指定size时)
        y = np.random.randint(0, 1 + w_old - w)
        z = np.random.randint(0, 1 + d_old - d)
    else: #如果 flag 不是 'train'，则进行中心裁剪。计算裁剪起始点 (x, y, z) 使得裁剪的区域在图像中居中。
        # crop center
        x = int((h_old - h) / 2)
        y = int((w_old - w) / 2)
        z = int((d_old - d) / 2)

    return self.do_crop_3d(image, x, y, z, h, w, d)

def do_crop_3d(self, image, x, y, z, h, w, d): # 3D裁剪的辅助函数，确保整数和返回对应位置的3D块
    assert type(x) == int, x
    assert type(y) == int, y
    assert type(z) == int, z
    assert type(h) == int, h
    assert type(w) == int, w
    assert type(d) == int, d

    return image[x:x + h, y:y + w, z:z + d] #这边取的时最边界点加上crop_size的大小，解释了为何上述代码的定中心方式如此

def crop_cubes_3d(self, image, flag, cubes_per_side, cube_jitter_xy=3, cube_jitter_z=3): #在3D图像中裁剪多个3D立方体的方法，jitter貌似是为了防止数据连续变化留出的间隔大小，从代码里可以看出如果有jitter则再次进行3D裁切，cubes_per_side应该是每个方向有多少个cube
    h, w, d = image.shape

    patch_overlap = -cube_jitter_xy if cube_jitter_xy < 0 else 0 #patch_overlap 计算了在裁剪时可能存在的重叠区域。如果 cube_jitter_xy 是负数，则 patch_overlap 采用该值；否则，为零。

    #h_grid、w_grid 和 d_grid 计算了每个立方体的网格大小，以确保裁剪时没有重叠区域。
    # 这里主要思想应该是把切出来的3D大块划分为各个小块,这里拿笔记做做数学题吧，好久没写了。
    h_grid = (h - patch_overlap) // cubes_per_side
    w_grid = (w - patch_overlap) // cubes_per_side
    d_grid = (d - patch_overlap) // cubes_per_side
    h_patch = h_grid - cube_jitter_xy
    w_patch = w_grid - cube_jitter_xy
    d_patch = d_grid - cube_jitter_z

    cubes = []
    for i in range(cubes_per_side):
        for j in range(cubes_per_side):
            for k in range(cubes_per_side):

                p = self.do_crop_3d(image, #当i=0时，从0截取到第一个grid
                                i * h_grid,
                                j * w_grid,
                                k * d_grid,
                                h_grid + patch_overlap,
                                w_grid + patch_overlap,
                                d_grid + patch_overlap)

                if h_patch < h_grid or w_patch < w_grid or d_patch < d_grid: #如果有jitter，则对之前裁切好的3D块再次裁切以保留jitter大小的间隔
                    p = self.crop_3d(p, flag, [h_patch, w_patch, d_patch])

                cubes.append(p)

    return cubes

def rearrange(self, cubes, K_permutations):  # ...根据排列重新排列立方体的方法
    label = random.randint(0, len(K_permutations) - 1)
    # print('label', np.array(K_permutations[label]), label)
    # print(cubes[np.array(K_permutations[label])])
    return np.array(cubes)[np.array(K_permutations[label])], label

def center_crop_xy(self, image, size): # 在image中间截一个size大小的缺口，画图很容易理解
    """CenterCrop a sample.
        Args:
            image: [D, H, W]
            label:[D, H, W]
            crop_size: the desired output size in the x-y plane
        Returns:
            out_image:[D, h, w]
            out_label:[D, h, w]
    """
    h, w, d = image.shape

    h1 = int(round((h - size[0]) / 2.)) #round() 函数将结果四舍五入为最接近的整数，确保裁剪区域的起始点是整数。
    w1 = int(round((w - size[1]) / 2.))

    image = image[h1:h1 + size[0], w1:w1 + size[1], :]
    return image

def rotate(self, cubes): # ...（旋转3D立方体的方法）

    # multi-hot labels
    # [8, H, W, D]
    rot_cubes = copy.deepcopy(cubes) # 创建输入立方体的深层副本，以免修改原始数据
    hor_vector = [] # 记录水平旋转的向量
    ver_vector = [] # 记录垂直旋转的向量

    for i in range(self.num_cubes):
        p = random.random()  # 生成一个0到1之间的随机数
        cube = rot_cubes[i] # 获取当前处理的立方体,这里debug看cube是个tensor，后续会出bug
        # cube_ = cube.numpy() #设置一个numpy_arrary防止np.flip bug, 如果输入是nparray就不用这步
        # p = 0.1 # for test
        # [H, W, D]
        if p < 1/3: # 如果 p 小于 1/3，表示进行水平旋转。将水平旋转的标志添加到 hor_vector，并沿x轴翻转180度。
            hor_vector.append(1)
            ver_vector.append(0)
            # rotate 180 along x axis
            rot_cubes[i] = np.flip(cube, (1, 2)) # 沿x轴翻转180度,BUG ValueError: step must be greater than zero,最后输出的应该是numpy形式
            # cube = np.flip(cube_, (1, 2))
            # rot_cubes[i] = cube
        elif p < 2/3: #如果 p 大于等于 1/3 且小于 2/3，表示进行垂直旋转。将垂直旋转的标志添加到 ver_vector，并沿z轴翻转180度。
            hor_vector.append(0)
            ver_vector.append(1)
            # rotate 180 along z axis
            rot_cubes[i] = np.flip(cube, (0, 1)) # 沿z轴翻转180度
            # cube = np.flip(cube_, (0, 1))
            # rot_cubes[i] = cube

        else: #如果 p 大于等于 2/3，不进行旋转，标志都设为0。
            hor_vector.append(0)
            ver_vector.append(0)

    return rot_cubes, hor_vector, ver_vector

def mask(self, cubes): # ...（对3D立方体应用掩码的方法）
    mask_vector = []
    masked_cubes = copy.deepcopy(cubes) # 创建一个输入立方体的深层副本，以免修改原始数据
    for i in range(self.num_cubes):
        cube = masked_cubes[i] # 获取当前处理的立方体
        if random.random() < 0.5: # 如果随机数小于0.5，应用掩码
            # mask
            mask_vector.append(1) # 记录掩码的标志为1
            R = np.random.uniform(0, 1, cube.shape) # 生成与立方体相同形状的随机数矩阵
            R = (R > 0.5).astype(np.int32) # 将大于0.5的值设置为1，否则为0，形成二值掩码
            masked_cubes[i] = cube * R # 将立方体与二值掩码相乘，进行掩码操作
        else:
            mask_vector.append(0) # 如果随机数大于等于0.5，不应用掩码，标志为0

    return masked_cubes, mask_vector

测试simpleVIT

In [1]:
# input为[b, 1, 240, 240, 154]

from vit_pytorch import SimpleViT
import torch

all_cubes = torch.randn([4, 1, 240, 240, 154])
vit = SimpleViT(image_size = 240,patch_size = 120,num_classes = 1024,dim = 1024,depth = 1,heads = 1,mlp_dim = 2048,channels = 1)

In [2]:
output = vit(all_cubes)

EinopsError: Expected 4 dimensions, got 5